<a href="https://colab.research.google.com/github/RefineX/Change-Detection/blob/main/AML_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing libraries & mounting google drive

In [ ]:
!pip install image-classifiers
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.9 MB/s 
Mounted at /content/drive


In [ ]:
#@title Data Downloader
dataset = "LEVIRCD_Plus" #@param ["S2Looking", "LEVIRCD_Plus"]
resize = "512" #@param ["1024", "512", "256"]
crop = "256" #@param ["1024", "512", "256"]

data_link_dict = {
    'S2Looking': {
      '1024_512': "1-0_vcODMYmyYIY_uhMcs97aAk2OJVduq&confirm=t",
      '1024_256': "1-0mVd6BjKnG3LXhYkbyxdx0-dtRKY6KO&confirm=t",
      '512_512': "1-6WOmE0LTJ4Z31EphmA0L8QY5NMMllsT&confirm=t",
      '512_256': "1-7NSOmHTsDpkEtbgt_N4VSgipCo5kr3a&confirm=t",
      '256_256': "1-8x5d5DrNsgJ5eAOYyKwi91g8knGeRjQ&confirm=t",
      '1024_1024': "1GzrgMwJKguXSWSfFsBSC2qSr52fVEc7W&confirm=t"
    },
    'LEVIRCD_Plus': {
      '1024_512': "1-22GjfF8mlLFNyoJaa6_GMJRfTCoGZyc&confirm=t",
      '1024_256': "1-2r-zCCfQjLRtwwXMGPYAxEJcTS8Hp80&confirm=t",
      '512_512': "1-43scZrxe3Q_PH2EnBRuc_6l9O9WjPs2&confirm=t",
      '512_256': "1-5oC0xV36S4K5VMiQuwheWoyMt1ymYb9&confirm=t",
      '256_256': "1-69cdgqlcXunt5vrCR9jRvcMkpPNzYWr&confirm=t",
      '1024_1024': "1nyPJZGGOL7o8A0m0rGw7wyu2BFIxC4SD&confirm=t"
    }
}

import os
if os.path.exists(f'Data/{dataset}/{resize}_{crop}'):
  print('This dataset already exists.')
else:
  gdown_link = data_link_dict[dataset][f'{resize}_{crop}']
  !gdown "{gdown_link}"
  print('Unzipping...',end='')
  !unzip -q "{resize}_{crop}.zip"
  print('Done.\nDeleting zip...',end='')
  !rm "{resize}_{crop}.zip"
  print('Done.')

Downloading...
From: https://drive.google.com/uc?id=1-5oC0xV36S4K5VMiQuwheWoyMt1ymYb9&confirm=t
To: /content/512_256.zip
100% 317M/317M [00:02<00:00, 127MB/s]
Unzipping...Done.
Deleting zip...Done.


## Importing dependencies

In [ ]:
# Import libraries
import tensorflow as tf
import os
import time
import numpy as np
from tqdm.notebook import tqdm
from pathlib import Path
from matplotlib import pyplot as plt
from IPython import display
import cv2
import pywt
import random
import shutil as sh

In [ ]:
#@title Dataset Characteristics
dataset = "LEVIRCD_Plus" #@param ["S2Looking", "LEVIRCD_Plus"]
resized_size = "512" #@param [1024, 512, 256]
crop_size = "256" #@param [1024, 512, 256]
resized_size = int(resized_size)
crop_size = int(crop_size)
if dataset == 'S2Looking':
  pre_image = 'Image1'
  post_image = 'Image2'
elif dataset == 'LEVIRCD_Plus':
  pre_image = 'A'
  post_image = 'B'
label_image = 'label'
PATH = f"/content/Data/{dataset}/{resized_size}_{crop_size}"
Path('saved/models/').mkdir(parents=True, exist_ok=True)
Path('saved/histories/').mkdir(parents=True, exist_ok=True)

## Subset selection for few-shot learning

In [ ]:
# Subset Choice
subset_percent = "10" #@param ["10", "25", "50", "100"]
if subset_percent == "100":
  PATH = PATH
else:
  Subset_PATH = PATH+"_subset_{}".format(subset_percent)
  Path(Subset_PATH).mkdir(parents=True, exist_ok=True)
  dirs = os.listdir(PATH)
  subdirs = os.listdir(PATH+"/"+dirs[0])
  for dir in dirs:
    for subdir in subdirs:
      Path(Subset_PATH+"/"+dir+"/"+subdir).mkdir(parents=True, exist_ok=True)

  for dir in dirs:
    img_list = os.listdir(PATH+"/"+dir+"/"+pre_image)
    initial_count = len(img_list)
    final_count = int(initial_count * int(subset_percent)/100)
    random.seed(0)
    subset_img_list = random.sample(img_list, final_count)
    for subdir in subdirs:
      for subset_img in subset_img_list:
        if subdir == label_image:
          source_path = PATH+"/"+dir+"/"+subdir+"/"+subset_img.replace('jpg','png')
          destination_path = Subset_PATH+"/"+dir+"/"+subdir+"/"+subset_img.replace('jpg','png')
        else:
          source_path = PATH+"/"+dir+"/"+subdir+"/"+subset_img
          destination_path = Subset_PATH+"/"+dir+"/"+subdir+"/"+subset_img
        sh.copy(source_path, destination_path)
  PATH = Subset_PATH

## Defining metrics

In [ ]:
# Metrics

def iou(y_true,y_pred):
  y_pred = tf.argmax(y_pred,-1)
  y_true = tf.argmax(y_true,-1)
  tp = tf.cast(tf.reduce_sum(y_pred*y_true,axis=[1,2]), 'float32')
  fp = tf.cast(tf.reduce_sum(y_pred*(1-y_true),axis=[1,2]), 'float32')
  fn = tf.cast(tf.reduce_sum((1-y_pred)*y_true,axis=[1,2]), 'float32')
  iou_value = (tp + 1e-14) / (tp + fp + fn + 1e-14)
  return tf.reduce_mean(iou_value)

def miou(y_true,y_pred):
  y_pred = tf.argmax(y_pred,-1)
  y_true = tf.argmax(y_true,-1)
  tp = tf.cast(tf.reduce_sum(y_pred*y_true,axis=[1,2]), 'float32')
  tn = tf.cast(tf.reduce_sum((1-y_pred)*(1-y_true),axis=[1,2]), 'float32')
  fp = tf.cast(tf.reduce_sum(y_pred*(1-y_true),axis=[1,2]), 'float32')
  fn = tf.cast(tf.reduce_sum((1-y_pred)*y_true,axis=[1,2]), 'float32')
  iou1 = (tp + 1e-14) / (tp + fp + fn + 1e-14)
  iou2 = (tn + 1e-14) / (tn + fp + fn + 1e-14)
  iou_value = (iou1 + iou2) / 2
  return tf.reduce_mean(iou_value)

# Losses

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from classification_models.tfkeras import Classifiers
import pickle
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:
epochs = 50
es_patience = 15
rLR_patience = 5
lr = 1e-2
loss = 'categorical_crossentropy'
BATCH_SIZE = 16
metrics = [iou, 'accuracy']

# Add Augmentations

## Canny

In [ ]:
# Helper Functions for Model #1

def canny_edge1(image):
  l,u = 100, 200
  edgesA = cv2.Canny(image[:,:,:3].numpy()[:,:,::-1], l, u)
  edgesB = cv2.Canny(image[:,:,3:].numpy()[:,:,::-1], l, u)
  edges = tf.stack([edgesA,edgesB],axis=-1)
  edges = tf.cast(edges,tf.float32)
  edges /= tf.reduce_max(edges)
  return edges

def canny_edge2(image):
  l,u = 100, 200
  eA = cv2.Canny(image[:,:,:3].numpy()[:,:,::-1], l, u) / 2.01
  eB = cv2.Canny(image[:,:,3:].numpy()[:,:,::-1], l, u) / 2.01
  e = eA-eB

  edgesA = e>0
  edgesB = e<0
  edges = tf.stack([edgesA,edgesB],axis=-1)
  edges = tf.cast(edges,tf.float32)
  edges /= tf.reduce_max(edges)
  return edges


def load(imageA_file):
  imageA = tf.io.read_file(imageA_file)
  imageA = tf.image.decode_png(imageA)[:,:,:3]

  imageB_file = tf.strings.regex_replace(imageA_file, f'/{pre_image}/', f'/{post_image}/')
  imageB = tf.io.read_file(imageB_file)
  imageB = tf.image.decode_jpeg(imageB)[:,:,:3]

  label_file = tf.strings.regex_replace(imageA_file, f'/{pre_image}/', f'/{label_image}/')
  label_file = tf.strings.regex_replace(label_file, '.jpg', '.png')
  label = tf.io.read_file(label_file)
  label = tf.image.decode_png(label)[:,:,0]

  image = tf.concat([imageA,imageB],axis=-1)
  edges = tf.py_function(canny_edge1, [image], tf.float32)
  image = tf.cast(image,tf.float32)
  label = tf.cast(label,tf.float32)
  label = tf.stack([255-label,label],axis=-1)

  return image, edges, label

def normalize(image, label):
  image = image / 255
  label = label / 255
  return image, label

def load_image(image_file):
  image, edges, label = load(image_file)
  image, label = normalize(image, label)
  image = tf.concat([image,edges],axis=-1)
  return image, label

train_dataset = tf.data.Dataset.list_files(f'{PATH}/train/{pre_image}/*.jpg')
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
train_dataset_length = len(train_dataset)
# train_dataset_length = int(np.ceil(len(os.listdir(f'{PATH}/train/{pre_image}/'))/BATCH_SIZE))

val_dataset = tf.data.Dataset.list_files(f'{PATH}/val/{pre_image}/*.jpg')
val_dataset = val_dataset.map(load_image)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset_length = len(val_dataset)
# val_dataset_length = int(np.ceil(len(os.listdir(f'{PATH}/train/{pre_image}/'))/BATCH_SIZE))

test_dataset = tf.data.Dataset.list_files(f'{PATH}/test/{pre_image}/*.jpg')
test_dataset = test_dataset.map(load_image)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset_length = len(test_dataset)
# test_dataset_length = len(os.listdir(f'{PATH}/train/{pre_image}/'))

print(f'train_dataset_length: {train_dataset_length}, val_dataset_length: {val_dataset_length}, test_dataset_length: {test_dataset_length}')

In [ ]:
# # Dataset test
# for x,y in train_dataset.take(1):
#   pass
# idx = 5
# plt.figure(figsize=(20,4))
# plt.subplot(151)
# plt.imshow(x[idx,:,:,:3].numpy())
# plt.subplot(152)
# plt.imshow(x[idx,:,:,6].numpy(),cmap='gray')
# plt.subplot(153)
# plt.imshow(x[idx,:,:,3:6].numpy())
# plt.subplot(154)
# plt.imshow(x[idx,:,:,7].numpy(),cmap='gray')
# plt.subplot(155)
# plt.imshow(y[idx].numpy().argmax(-1),cmap='gray')

In [ ]:
# Model #1
def ef_model(input_shape = (256, 256, 8), backbone = 'resnet18'):

  ResNet18, preprocess_input = Classifiers.get(backbone)
  encoder = ResNet18(input_shape, weights=None,include_top=False)
  encoder_output = encoder.output
  skip_outputs = [encoder.get_layer(f).output for f in ['stage4_unit1_relu1', 'stage3_unit1_relu1', 'stage2_unit1_relu1', 'relu0']]

  def conv_bn_relu(filters,x):
    x = Conv2D(filters, 3, padding='same', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

  x = encoder_output
  for i,skip_output in enumerate(skip_outputs):
    x = UpSampling2D(name=f'up{i+1}')(x)
    filters = skip_output.shape[-1]
    x = conv_bn_relu(filters,x)
    x = conv_bn_relu(filters,x)
    x = concatenate([x, skip_output])
  x = UpSampling2D(name='up5')(x)
  for filters in [32,32,16,16]:
    x = conv_bn_relu(filters,x)
  x = Conv2D(2, 1, activation = 'softmax', padding='same', kernel_initializer='he_uniform', dtype = 'float32')(x)
  model = Model(inputs = encoder.inputs, outputs = x)
  return model

In [ ]:
# Model #1
input_shape = (crop_size, crop_size, 8)
backbone = 'resnet18'
model = ef_model(input_shape, backbone)
model_name = 'model_ef'
model_path = f'saved/models/{dataset}_{backbone}_{model_name}_canny1.h5'
history_path = f'saved/histories/{dataset}_{backbone}_{model_name}_canny1.pkl'


checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_iou', mode='max', save_best_only=True, verbose = 1)
reduce_lr = ReduceLROnPlateau(monitor='val_iou', mode = 'max', factor=1/np.sqrt(10), patience = rLR_patience, min_lr=1e-6, verbose = 1)
earlystopper = EarlyStopping(monitor='val_iou', mode = 'max', patience = es_patience, verbose=1)
callbacks = [checkpoint, reduce_lr, earlystopper]
optimizer = Adam(learning_rate = lr)
model.compile(optimizer = optimizer, loss = loss, metrics = metrics)
hist = model.fit(train_dataset, epochs = epochs, validation_data = val_dataset, callbacks = callbacks)
with open(history_path, 'wb') as file_pi:
  pickle.dump(hist.history, file_pi)

model.load_weights(model_path)
_, test_iou, _ = model.evaluate(test_dataset)
os.rename(model_path, model_path.replace('.h5',f'_{test_iou:.4f}.h5'))
os.rename(history_path, history_path.replace('.pkl',f'_{test_iou:.4f}.pkl'))
plt.plot(hist.history['iou'])
plt.plot(hist.history['val_iou'])
plt.title('model iou')
plt.ylabel('iou')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

## MRA

In [ ]:
# Helper Functions for Model #1

def mra_haar(image):

  decomp = pywt.wavedec2(cv2.cvtColor(image[:,:,:3].numpy(),cv2.COLOR_RGB2GRAY),wavelet='haar',mode='constant',level=1)
  h_dA = decomp[1][0]
  h_dA = (h_dA-h_dA.min())/(h_dA.max()-h_dA.min())
  h_dA = cv2.resize(h_dA, (crop_size,crop_size), cv2.INTER_LINEAR)
  v_dA = decomp[1][1]
  v_dA = (v_dA-v_dA.min())/(v_dA.max()-v_dA.min())
  v_dA = cv2.resize(v_dA, (crop_size,crop_size), cv2.INTER_LINEAR)
  d_dA = decomp[1][2]
  d_dA = (d_dA-d_dA.min())/(d_dA.max()-d_dA.min())
  d_dA = cv2.resize(d_dA, (crop_size,crop_size), cv2.INTER_LINEAR)

  decomp = pywt.wavedec2(cv2.cvtColor(image[:,:,3:].numpy(),cv2.COLOR_RGB2GRAY),wavelet='haar',mode='constant',level=1)
  h_dB = decomp[1][0]
  h_dB = (h_dB-h_dB.min())/(h_dB.max()-h_dB.min())
  h_dB = cv2.resize(h_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  v_dB = decomp[1][1]
  v_dB = (v_dB-v_dB.min())/(v_dB.max()-v_dB.min())
  v_dB = cv2.resize(v_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  d_dB = decomp[1][2]
  d_dB = (d_dB-d_dB.min())/(d_dB.max()-d_dB.min())
  d_dB = cv2.resize(d_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  edges = tf.stack([h_dA, v_dA, d_dA, h_dB, v_dB, d_dB],axis=-1)
  edges = tf.cast(edges,tf.float32)

  return edges

def mra_db7(image):

  decomp = pywt.wavedec2(cv2.cvtColor(image[:,:,:3].numpy(),cv2.COLOR_RGB2GRAY),wavelet='db7',mode='constant',level=1)
  h_dA = decomp[1][0]
  h_dA = (h_dA-h_dA.min())/(h_dA.max()-h_dA.min())
  h_dA = cv2.resize(h_dA, (crop_size,crop_size), cv2.INTER_LINEAR)
  v_dA = decomp[1][1]
  v_dA = (v_dA-v_dA.min())/(v_dA.max()-v_dA.min())
  v_dA = cv2.resize(v_dA, (crop_size,crop_size), cv2.INTER_LINEAR)
  d_dA = decomp[1][2]
  d_dA = (d_dA-d_dA.min())/(d_dA.max()-d_dA.min())
  d_dA = cv2.resize(d_dA, (crop_size,crop_size), cv2.INTER_LINEAR)

  decomp = pywt.wavedec2(cv2.cvtColor(image[:,:,3:].numpy(),cv2.COLOR_RGB2GRAY),wavelet='db7',mode='constant',level=1)
  h_dB = decomp[1][0]
  h_dB = (h_dB-h_dB.min())/(h_dB.max()-h_dB.min())
  h_dB = cv2.resize(h_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  v_dB = decomp[1][1]
  v_dB = (v_dB-v_dB.min())/(v_dB.max()-v_dB.min())
  v_dB = cv2.resize(v_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  d_dB = decomp[1][2]
  d_dB = (d_dB-d_dB.min())/(d_dB.max()-d_dB.min())
  d_dB = cv2.resize(d_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  edges = tf.stack([h_dA, v_dA, d_dA, h_dB, v_dB, d_dB],axis=-1)
  edges = tf.cast(edges,tf.float32)

  return edges


def load(imageA_file):
  imageA = tf.io.read_file(imageA_file)
  imageA = tf.image.decode_png(imageA)[:,:,:3]

  imageB_file = tf.strings.regex_replace(imageA_file, f'/{pre_image}/', f'/{post_image}/')
  imageB = tf.io.read_file(imageB_file)
  imageB = tf.image.decode_jpeg(imageB)[:,:,:3]

  label_file = tf.strings.regex_replace(imageA_file, f'/{pre_image}/', f'/{label_image}/')
  label_file = tf.strings.regex_replace(label_file, '.jpg', '.png')
  label = tf.io.read_file(label_file)
  label = tf.image.decode_png(label)[:,:,0]

  image = tf.concat([imageA,imageB],axis=-1)
  edges = tf.py_function(mra_haar, [image], tf.float32)
  image = tf.cast(image,tf.float32)
  label = tf.cast(label,tf.float32)
  label = tf.stack([255-label,label],axis=-1)

  return image, edges, label

def normalize(image, label):
  image = image / 255
  label = label / 255
  return image, label

def load_image(image_file):
  image, edges, label = load(image_file)
  image, label = normalize(image, label)
  image = tf.concat([image,edges],axis=-1)
  return image, label

train_dataset = tf.data.Dataset.list_files(f'{PATH}/train/{pre_image}/*.jpg')
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
train_dataset_length = len(train_dataset)
# train_dataset_length = int(np.ceil(len(os.listdir(f'{PATH}/train/{pre_image}/'))/BATCH_SIZE))

val_dataset = tf.data.Dataset.list_files(f'{PATH}/val/{pre_image}/*.jpg')
val_dataset = val_dataset.map(load_image)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset_length = len(val_dataset)
# val_dataset_length = int(np.ceil(len(os.listdir(f'{PATH}/train/{pre_image}/'))/BATCH_SIZE))

test_dataset = tf.data.Dataset.list_files(f'{PATH}/test/{pre_image}/*.jpg')
test_dataset = test_dataset.map(load_image)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset_length = len(test_dataset)
# test_dataset_length = len(os.listdir(f'{PATH}/train/{pre_image}/'))

print(f'train_dataset_length: {train_dataset_length}, val_dataset_length: {val_dataset_length}, test_dataset_length: {test_dataset_length}')

train_dataset_length: 128, val_dataset_length: 32, test_dataset_length: 87


In [ ]:
# # Dataset test
# for x,y in train_dataset.take(1):
#   pass
# idx = 5
# plt.figure(figsize=(40,16))
# plt.subplot(251)
# plt.imshow(x[idx,:,:,:3].numpy())
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(252)
# plt.imshow(x[idx,:,:,6].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(256)
# plt.imshow(x[idx,:,:,7].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(257)
# plt.imshow(x[idx,:,:,8].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(253)
# plt.imshow(x[idx,:,:,3:6].numpy())
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(254)
# plt.imshow(x[idx,:,:,9].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(258)
# plt.imshow(x[idx,:,:,10].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(259)
# plt.imshow(x[idx,:,:,11].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(255)
# plt.imshow(y[idx].numpy().argmax(-1),cmap='gray')
# plt.axis('off')
# plt.tight_layout()

In [ ]:
# Model #1
def ef_model(input_shape = (256, 256, 8), backbone = 'resnet18'):

  ResNet18, preprocess_input = Classifiers.get(backbone)
  encoder = ResNet18(input_shape, weights=None,include_top=False)
  encoder_output = encoder.output
  skip_outputs = [encoder.get_layer(f).output for f in ['stage4_unit1_relu1', 'stage3_unit1_relu1', 'stage2_unit1_relu1', 'relu0']]

  def conv_bn_relu(filters,x):
    x = Conv2D(filters, 3, padding='same', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

  x = encoder_output
  for i,skip_output in enumerate(skip_outputs):
    x = UpSampling2D(name=f'up{i+1}')(x)
    filters = skip_output.shape[-1]
    x = conv_bn_relu(filters,x)
    x = conv_bn_relu(filters,x)
    x = concatenate([x, skip_output])
  x = UpSampling2D(name='up5')(x)
  for filters in [32,32,16,16]:
    x = conv_bn_relu(filters,x)
  x = Conv2D(2, 1, activation = 'softmax', padding='same', kernel_initializer='he_uniform', dtype = 'float32')(x)
  model = Model(inputs = encoder.inputs, outputs = x)
  return model

In [ ]:
# Model #1
input_shape = (crop_size, crop_size, 12)
backbone = 'resnet18'
model = ef_model(input_shape, backbone)
model_name = 'model_ef'
model_path = f'saved/models/{dataset}_{backbone}_{model_name}_mra_haar.h5'
history_path = f'saved/histories/{dataset}_{backbone}_{model_name}_mra_haar.pkl'


checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_iou', mode='max', save_best_only=True, verbose = 1)
reduce_lr = ReduceLROnPlateau(monitor='val_iou', mode = 'max', factor=1/np.sqrt(10), patience = rLR_patience, min_lr=1e-6, verbose = 1)
earlystopper = EarlyStopping(monitor='val_iou', mode = 'max', patience = es_patience, verbose=1)
callbacks = [checkpoint, reduce_lr, earlystopper]
optimizer = Adam(learning_rate = lr)
model.compile(optimizer = optimizer, loss = loss, metrics = metrics)
hist = model.fit(train_dataset, epochs = epochs, validation_data = val_dataset, callbacks = callbacks)
with open(history_path, 'wb') as file_pi:
  pickle.dump(hist.history, file_pi)

model.load_weights(model_path)
_, test_iou, _ = model.evaluate(test_dataset)
os.rename(model_path, model_path.replace('.h5',f'_{test_iou:.4f}.h5'))
os.rename(history_path, history_path.replace('.pkl',f'_{test_iou:.4f}.pkl'))
plt.plot(hist.history['iou'])
plt.plot(hist.history['val_iou'])
plt.title('model iou')
plt.ylabel('iou')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

# Move saved files to drive

In [ ]:
!zip -r saved.zip saved/
!cp saved.zip /content/drive/MyDrive/saved.zip